In [1]:
import subprocess

In [28]:
def run_fssp(instance, init=0, improvement=0, neighbourhood=0):
    init = "--random-init" if not init else "--srz"
    improvement = "--first" if not improvement else "--best"
    neighbourhood = "--transpose" if neighbourhood == 0 else "--exchange" if neighbourhood == 1 else "--insert"
    command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
    output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
    solution, time = output.decode('utf-8').split('\n')[:2]
    return command, [int(x) for x in solution.split(" ") if x], int(time)


In [31]:
for init in [0, 1]:
    for improvement in [0, 1]:
        for neighbourhood in [0, 1, 2]:
            command, sol, time = run_fssp("../instances/100_20_01", init, improvement, neighbourhood)
            print(command)
            print(time)

./fssp ../instances/100_20_01 --random-init --first --transpose
2469222
./fssp ../instances/100_20_01 --random-init --first --exchange
1816290
./fssp ../instances/100_20_01 --random-init --first --insert
1957238
./fssp ../instances/100_20_01 --random-init --best --transpose
2493326
./fssp ../instances/100_20_01 --random-init --best --exchange
1845924
./fssp ../instances/100_20_01 --random-init --best --insert
1983587
./fssp ../instances/100_20_01 --srz --first --transpose
1861380
./fssp ../instances/100_20_01 --srz --first --exchange
1841685
./fssp ../instances/100_20_01 --srz --first --insert
1829427
./fssp ../instances/100_20_01 --srz --best --transpose
1861380
./fssp ../instances/100_20_01 --srz --best --exchange
1845234
./fssp ../instances/100_20_01 --srz --best --insert
1847052
